In [128]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col, count, round


sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Assignment_2_Batch")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

#  Google Storage File Path
airport_info_gsc_file_path = 'gs://dejads_a2_input_steven/AIRPORT_COORDINATES.csv'   
month_1_delay_gsc_file_path = 'gs://dejads_a2_input_steven/ONTIME_REPORTING_01.csv'   

In [129]:
# Create data frame load batch data - AIRPORT_COORDINATES.csv
df_airport = spark.read.format("csv").option("header", "true").load(airport_info_gsc_file_path)

In [130]:
# Create data frame load batch data - ONTIME_REPORTING_01.csv
df_flight = spark.read.format("csv").option("header", "true").load(month_1_delay_gsc_file_path)

In [131]:
df_airport.printSchema()
df_airport.show(truncate=False)

root
 |-- ORIGIN_AIRPORT_ID: string (nullable = true)
 |-- DISPLAY_AIRPORT_NAME: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)

+-----------------+------------------------+-----------+------------+
|ORIGIN_AIRPORT_ID|DISPLAY_AIRPORT_NAME    |LATITUDE   |LONGITUDE   |
+-----------------+------------------------+-----------+------------+
|10001            |Afognak Lake Airport    |58.10944444|-152.9066667|
|10003            |Bear Creek Mining Strip |65.54805556|-161.0716667|
|10004            |Lik Mining Camp         |68.08333333|-163.1666667|
|10005            |Little Squaw Airport    |67.57      |-148.1838889|
|10006            |Kizhuyak Bay            |57.74527778|-152.8827778|
|10007            |Klawock Seaplane Base   |55.55472222|-133.1016667|
|10008            |Elizabeth Island Airport|59.15694444|-151.8291667|
|10009            |Augustin Island         |59.36277778|-153.4305556|
|10010            |Columbia County        

In [132]:
df_flight.printSchema()
df_flight.show(truncate=False)

root
 |-- MONTH: string (nullable = true)
 |-- DAY_OF_MONTH: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- OP_UNIQUE_CARRIER: string (nullable = true)
 |-- TAIL_NUM: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: string (nullable = true)
 |-- ORIGIN_AIRPORT_ID: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY_NAME: string (nullable = true)
 |-- DEST_AIRPORT_ID: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_CITY_NAME: string (nullable = true)
 |-- CRS_DEP_TIME: string (nullable = true)
 |-- DEP_TIME: string (nullable = true)
 |-- DEP_DELAY_NEW: string (nullable = true)
 |-- DEP_DEL15: string (nullable = true)
 |-- DEP_TIME_BLK: string (nullable = true)
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- ARR_TIME: string (nullable = true)
 |-- ARR_DELAY_NEW: string (nullable = true)
 |-- ARR_TIME_BLK: string (nullable = true)
 |-- CANCELLED: string (nullable = true)
 |-- CANCELLATION_CODE: string (nullable 

In [133]:
# There exists some duplicated airport records
df_airport.select("ORIGIN_AIRPORT_ID").distinct().count() == df_airport.count()

False

In [134]:
# clever way to remove duplicated airport records
df_airport.drop_duplicates(['ORIGIN_AIRPORT_ID']).count()

# stupid way remove duplicated airport records
airport_window = Window.partitionBy('ORIGIN_AIRPORT_ID').orderBy('ORIGIN_AIRPORT_ID')

df_airport_clean = df_airport.withColumn('row_num', row_number().over(airport_window))\
                             .where(col('row_num') == 1)\
                             .drop('row_num')

In [135]:
# get the number of all flights in each airport
df_flight_count = df_flight.groupBy(col('ORIGIN_AIRPORT_ID').alias('AIRPORT_id'))\
                           .agg(count('*').alias('flight_count'))

# append to all flights info the name, latitude and longitude of the airport
df_flight_count = df_flight_count.join(df_airport_clean, col('AIRPORT_id') == col('ORIGIN_AIRPORT_ID'), 'left')

In [136]:
# select the delayed flights
df_delay = df_flight.where(df_flight.DEP_DELAY_NEW != 0).select("*")

# join flight delay table with airport info tale
joinExpression = df_delay["ORIGIN_AIRPORT_ID"] == df_airport_clean['ORIGIN_AIRPORT_ID']
df_delay_w_airport_name = df_delay.join(df_airport_clean, joinExpression,"left")

# checking
df_delay.count() == df_delay_w_airport_name.count()

True

In [137]:
# count delayed flight  for each airport
df_flight_summary = df_delay_w_airport_name.groupby('DISPLAY_AIRPORT_NAME')\
                                                .agg(count('*').alias('delayed_flight_count'))\
                                                .orderBy('delayed_flight_count', ascending=False)

In [138]:
# append to airport-delayed-flight table with detailed info of airport and all flights from airport-all-flights info table
df_flight_summary = df_flight_summary.alias('summary').join(df_flight_count.alias('ft_cnt'), 
                                                            col("summary.DISPLAY_AIRPORT_NAME") == col('ft_cnt.DISPLAY_AIRPORT_NAME'), 'left')\
                                                      .select([col('ft_cnt.ORIGIN_AIRPORT_ID')] +
                                                              [col('summary.' + column) for column in df_flight_summary.columns] + 
                                                              [col('ft_cnt.flight_count'), col('ft_cnt.LATITUDE'), col('ft_cnt.LONGITUDE')])\
                                                      .orderBy('delayed_flight_count', ascending=False)

# count the precentage of flight delay for each airport
df_flight_summary = df_flight_summary.withColumn('delay_ratio', col('delayed_flight_count') / col('flight_count')) \
                                     .select(col('ORIGIN_AIRPORT_ID'),
                                             col('DISPLAY_AIRPORT_NAME'),
                                             round(col('delay_ratio'), 3).alias('delay_ratio'),
                                             col('delayed_flight_count'),
                                             col('flight_count'),
                                             round(col('LATITUDE'), 2).alias('LATITUDE'),
                                             round(col('LONGITUDE'),2).alias('LONGITUDE'))\
                                     .orderBy('delay_ratio', ascending=False)

In [139]:
df_flight_summary.show(10, truncate=False)

+-----------------+-----------------------------------------------+-----------+--------------------+------------+--------+---------+
|ORIGIN_AIRPORT_ID|DISPLAY_AIRPORT_NAME                           |delay_ratio|delayed_flight_count|flight_count|LATITUDE|LONGITUDE|
+-----------------+-----------------------------------------------+-----------+--------------------+------------+--------+---------+
|12119            |Hagerstown Regional-Richard A. Henson Field    |1.0        |10                  |10          |39.71   |-77.73   |
|13832            |Ogdensburg International                       |0.818      |9                   |11          |44.68   |-75.47   |
|13829            |Ogden-Hinckley                                 |0.75       |6                   |8           |41.2    |-112.01  |
|12223            |Tri-State/Milton J. Ferguson Field             |0.7        |14                  |20          |38.37   |-82.56   |
|13983            |Owensboro Daviess County                       |0.

In [140]:
df_flight_summary.orderBy('delay_ratio').show(10, truncate=False)

+-----------------+--------------------------------------------+-----------+--------------------+------------+--------+---------+
|ORIGIN_AIRPORT_ID|DISPLAY_AIRPORT_NAME                        |delay_ratio|delayed_flight_count|flight_count|LATITUDE|LONGITUDE|
+-----------------+--------------------------------------------+-----------+--------------------+------------+--------+---------+
|10558            |William B. Heilig Field                     |0.057      |3                   |53          |41.88   |-103.6   |
|13127            |Lewiston Nez Perce County                   |0.094      |6                   |64          |46.37   |-117.01  |
|12899            |North Platte Regional Airport Lee Bird Field|0.094      |5                   |53          |41.13   |-100.7   |
|15074            |Searcy Field                                |0.097      |6                   |62          |36.16   |-97.09   |
|12177            |Lea County Hobbs                            |0.103      |6             

In [111]:
# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
bucket = "dejads_temp_yk"
spark.conf.set('temporaryGcsBucket', bucket)
# Saving the data to BigQuery
df_flight_summary.write.format('bigquery') \
  .option('table', 'jads-de-2021.assignment_2.flight_delays') \
  .mode("append") \
  .save()

In [125]:
# get the airport with the most delay rate
top_delay = df_flight_summary.limit(1)
top_delay.show(truncate=False)

# Saving the data to BigQuery
top_delay.write.format('bigquery') \
  .option('table', 'jads-de-2021.assignment_2.top_delay') \
  .mode("append") \
  .save()

+-----------------+-------------------------------------------+-----------+--------------------+------------+--------+---------+
|ORIGIN_AIRPORT_ID|DISPLAY_AIRPORT_NAME                       |delay_ratio|delayed_flight_count|flight_count|LATITUDE|LONGITUDE|
+-----------------+-------------------------------------------+-----------+--------------------+------------+--------+---------+
|12119            |Hagerstown Regional-Richard A. Henson Field|1.0        |10                  |10          |39.71   |-77.73   |
+-----------------+-------------------------------------------+-----------+--------------------+------------+--------+---------+



In [126]:
# get the airports with the top 2 least delay rates
top_2_least_delay = df_flight_summary.orderBy('delay_ratio').limit(2)
top_2_least_delay.show(truncate=False)
second_least_delay = top_2_least_delay.orderBy('delay_ratio', ascending=False).limit(1)
second_least_delay.show(truncate=False)

# Saving the data to BigQuery
second_least_delay.write.format('bigquery') \
  .option('table', 'jads-de-2021.assignment_2.second_least_delay') \
  .mode("append") \
  .save()

+-----------------+-------------------------+-----------+--------------------+------------+--------+---------+
|ORIGIN_AIRPORT_ID|DISPLAY_AIRPORT_NAME     |delay_ratio|delayed_flight_count|flight_count|LATITUDE|LONGITUDE|
+-----------------+-------------------------+-----------+--------------------+------------+--------+---------+
|10558            |William B. Heilig Field  |0.057      |3                   |53          |41.88   |-103.6   |
|13127            |Lewiston Nez Perce County|0.094      |6                   |64          |46.37   |-117.01  |
+-----------------+-------------------------+-----------+--------------------+------------+--------+---------+

+-----------------+-------------------------+-----------+--------------------+------------+--------+---------+
|ORIGIN_AIRPORT_ID|DISPLAY_AIRPORT_NAME     |delay_ratio|delayed_flight_count|flight_count|LATITUDE|LONGITUDE|
+-----------------+-------------------------+-----------+--------------------+------------+--------+---------+


In [141]:
# Stop the spark context
spark.stop()